In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
from google import colab

In [2]:
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)

Mounted at /content/drive


In [12]:
tracks_path = '/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/tracks.csv'
artists_path = '/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/artists.csv'

In [13]:
tracks = pd.read_csv(tracks_path)
artists = pd.read_csv(artists_path)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
print(tracks.columns)
print(artists.columns)

Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date'],
      dtype='object')
Index(['mbid', 'artist_mb', 'artist_lastfm', 'country_mb', 'country_lastfm',
       'tags_mb', 'tags_lastfm', 'listeners_lastfm', 'scrobbles_lastfm',
       'ambiguous_artist'],
      dtype='object')


In [ ]:
# Print number of NA values for each column in Artists
for col in artists.columns:
  print(col)
  print(artists[col].isna().sum())
  
# Drop all rows in Artists with missing mbid or artist_mb
print("-----")
artists = artists[~artists["artist_mb"].isna()]
for col in artists.columns:
  print(col)
  print(artists[col].isna().sum())

# Tracks has no missing (NA) values for any column

In [ ]:
diff = artists[artists["country_mb"] != artists["country_lastfm"]]
# print(diff[["artist_mb", "country_mb", "country_lastfm"]])

mbNA = artists[artists["country_mb"].isna() & ~artists["country_lastfm"].isna()]
# print(mbNA[["artist_mb", "country_mb", "country_lastfm"]])

fmNA = artists[~artists["country_mb"].isna() & artists["country_lastfm"].isna()]
print(fmNA[["artist_mb", "country_mb", "country_lastfm"]])

In [16]:
# Delete less useful columns
print(tracks.columns)
print("-")
print(artists.columns)

tracks.drop(["disc_number", "track_number", "time_signature"], inplace=True, axis=1)
tracks.drop("year", inplace=True, axis=1)
artists.drop("ambiguous_artist", inplace=True, axis=1)

Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date'],
      dtype='object')
-
Index(['mbid', 'artist_mb', 'artist_lastfm', 'country_mb', 'country_lastfm',
       'tags_mb', 'tags_lastfm', 'listeners_lastfm', 'scrobbles_lastfm',
       'ambiguous_artist'],
      dtype='object')


In [18]:
# Create Albums table
albums = tracks[["album", "album_id"]]
albums = albums.rename({'album': 'album_name'}, axis=1)

# # Combine country_mb and country_lastfm columns of Artists
artists["country"] = artists.apply(
    lambda row: row["country_mb"] if len(str(row["country_mb"])) > 0
                else row["country_lastfm"], axis=1)
artists["genres"] = artists.apply(
    lambda row: row["tags_mb"] if len(str(row["tags_mb"])) > 0
                else row["tags_lastfm"], axis=1)

# Create Artists table (rename columns)
artists = artists.rename({
    'mbid': 'artist_id',
    'artist_mb': 'artist_name',
    'listeners_lastfm': 'listeners',
    'scrobbles_lastfm': 'scrobbles'
}, axis=1)
artists.drop(["artist_lastfm", "country_mb", "country_lastfm", 
              "tags_mb", "tags_lastfm"], inplace=True, axis=1)

# Create Tracks table (rename columns)
tracks = tracks.rename({
    'id': 'track_id',
    'name': 'track_name',
    'artists': 'artist_names'
}, axis=1)

# Create SongBy table
songBy = tracks[["track_id", "artist_ids", "artist_names"]]
# Create AlbumBy table
albumBy = tracks[["album_id", "artist_ids"]]

# Drop columns from tracks
tracks.drop(["artist_names", "artist_ids", "album"], inplace=True, axis=1)
print(tracks.columns)


Index(['track_id', 'track_name', 'album', 'album_id', 'artist_names',
       'artist_ids', 'explicit', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'release_date'],
      dtype='object')


In [19]:
tracks = tracks.drop_duplicates()
albums = albums.drop_duplicates()
artists = artists.drop_duplicates()
albumBy = albumBy.drop_duplicates()
songBy = songBy.drop_duplicates()

In [20]:
print(tracks.columns)
print(albumBy.columns)
print(albumBy.head())

Index(['track_id', 'track_name', 'album', 'album_id', 'artist_names',
       'artist_ids', 'explicit', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'release_date'],
      dtype='object')
Index(['album_id', 'artist_ids'], dtype='object')
                  album_id                  artist_ids
0   2eia0myWFgoHuttJytCxgX  ['2d0hyoQ5ynDBnkvAbJKORj']
12  4LaRYkT4oy47wEuQgkLBul  ['2d0hyoQ5ynDBnkvAbJKORj']
22  4evw6IBex3N8x1oA2axMTH  ['77tT1kLj6mCWtFNqiOmP9H']
36  0eaPZK3nTV2Ei075j2t6Na  ['2U6gqwyl9F33YxawnFrZG7']
47  2yxoWG32Id6PzkER95aCtY  ['4sh4MHP7lhrSUakxwZzwqz']


In [30]:
# Explode albumBy - final
ab = albumBy[['album_id','artist_ids']]
ab['artist_ids'] = ab['artist_ids'].apply(lambda s: [id[1:-1] for id in s.replace(" ", "")[1:-1].split(',')])
print(len(ab))

ab_expl = ab.explode("artist_ids")
print(len(ab_expl))
ab_expl.drop_duplicates(inplace=True)
print(len(ab_expl))
ab_expl.rename({'artist_ids': 'artist_id'}, axis=1, inplace=True)
print(ab_expl.head())

print(ab_expl[ab_expl['album_id'].isnull()])

ab_expl.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/albumBy_expl.csv')

250260
457289
309774
                  album_id               artist_id
0   2eia0myWFgoHuttJytCxgX  2d0hyoQ5ynDBnkvAbJKORj
12  4LaRYkT4oy47wEuQgkLBul  2d0hyoQ5ynDBnkvAbJKORj
22  4evw6IBex3N8x1oA2axMTH  77tT1kLj6mCWtFNqiOmP9H
36  0eaPZK3nTV2Ei075j2t6Na  2U6gqwyl9F33YxawnFrZG7
47  2yxoWG32Id6PzkER95aCtY  4sh4MHP7lhrSUakxwZzwqz


In [ ]:
print(tracks.columns)
print(artists.columns)
print(albums.columns)

print(songBy.columns)
print(albumBy.columns)

In [ ]:
# Split songBy into one row per artist in list of artists
# sb = songBy[['track_id','artist_ids']]
sb = songBy[songBy.columns]
sb['artist_ids'] = sb['artist_ids'].apply(lambda s: [id[1:-1] for id in s.replace(" ", "")[1:-1].split(',')])
sb['artist_names'] = sb['artist_names'].apply(lambda s: [id[1:-1] for id in s.replace(" ", "")[1:-1].split(',')])
# sb = sb.set_index('track_id').apply(pd.Series.explode).reset_index()
# sb_linkexpl = sb.set_index('track_id').apply(pd.Series.explode)#.reset_index()

# print(sb_linkexpl.iloc[305:310])
# print(len(sb_linkexpl))

# print(sb.iloc[298:303])
# print(sb.loc[(sb['artist_ids'].size() == sb['artist_names'].size())].head(30))
# print(len(sb))
# sb_expl = sb.explode(['artist_ids','artist_names'])
# print(len(sb_expl))
# print(sb_expl.head())
# sb_expl = sb_expl.explode('artist_names')
# print(len(sb_expl))

# sb.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/songByDf_ids_only_split.csv')

In [ ]:
# Explode with for loop
rowDictList = []
for index, row in sb.iterrows():
  for i in range(len(row['artist_ids'])):
    rowDict = {
        'track_id': row['track_id'],
        'artist_id': row['artist_ids'][i],
        'artist_name': row['artist_names'][i]
    }
    rowDictList.append(rowDict)
sb_expl = pd.DataFrame(rowDictList)

In [ ]:
# Final SongBy
sb_art = sb_expl.merge(artists, on='artist_name', how='inner')
print(sb_art.head())
print(len(sb_art))
sb_art_intersect = sb_art[['track_id','artist_id_x']]
sb_art_intersect.rename({'artist_id_x':'artist_id'}, axis=1, inplace=True)



sb_art_intersect.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/songby_new.csv')

                 track_id             artist_id_x artist_name  \
0  45ZjRrEFgHJhCQ72qKDeem  0HjoyITAvSVktTCjXUa40Q       Rufio   
1  4gd1XbH3adOCB078KP7Dbd  0HjoyITAvSVktTCjXUa40Q       Rufio   
2  48LAwhnuZb4un5EdUluL8K  0HjoyITAvSVktTCjXUa40Q       Rufio   
3  4rP96apbJCohHkJB4aiBAf  0HjoyITAvSVktTCjXUa40Q       Rufio   
4  1RqJYBleL85OXwqO8Ri82I  0HjoyITAvSVktTCjXUa40Q       Rufio   

                            artist_id_y  listeners  scrobbles        country  \
0  19d37d2f-fefc-4ff3-aeb1-1aeae97ea6de   300129.0  5333939.0  United States   
1  19d37d2f-fefc-4ff3-aeb1-1aeae97ea6de   300129.0  5333939.0  United States   
2  19d37d2f-fefc-4ff3-aeb1-1aeae97ea6de   300129.0  5333939.0  United States   
3  19d37d2f-fefc-4ff3-aeb1-1aeae97ea6de   300129.0  5333939.0  United States   
4  19d37d2f-fefc-4ff3-aeb1-1aeae97ea6de   300129.0  5333939.0  United States   

                  genres  
0  punk; usa; california  
1  punk; usa; california  
2  punk; usa; california  
3  punk; usa; califo

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
print(len(sb_art_intersect))
sb_art_intersect.drop_duplicates(inplace=True)
sb_art_intersect['artist_id'] = sb_art_intersect['artist_id'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
sb_art_intersect['track_id'] = sb_art_intersect['track_id'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
print(len(sb_art_intersect))
print(sb_art_intersect.head())

234311


/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
<ipython-input-133-9c5237271ec1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sb_art_intersect['artist_id'] = sb_art_intersect['artist_id'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


234311
                 track_id               artist_id
0  45ZjRrEFgHJhCQ72qKDeem  0HjoyITAvSVktTCjXUa40Q
1  4gd1XbH3adOCB078KP7Dbd  0HjoyITAvSVktTCjXUa40Q
2  48LAwhnuZb4un5EdUluL8K  0HjoyITAvSVktTCjXUa40Q
3  4rP96apbJCohHkJB4aiBAf  0HjoyITAvSVktTCjXUa40Q
4  1RqJYBleL85OXwqO8Ri82I  0HjoyITAvSVktTCjXUa40Q


<ipython-input-133-9c5237271ec1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sb_art_intersect['track_id'] = sb_art_intersect['track_id'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [ ]:
print(sb_art_intersect.head())
print(len(sb_art_intersect))
sb_art_intersect.drop_duplicates(inplace=True)
print(len(sb_art_intersect))
sb_art_intersect.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/songby_new.csv')


                 track_id               artist_id
0  45ZjRrEFgHJhCQ72qKDeem  0HjoyITAvSVktTCjXUa40Q
1  4gd1XbH3adOCB078KP7Dbd  0HjoyITAvSVktTCjXUa40Q
2  48LAwhnuZb4un5EdUluL8K  0HjoyITAvSVktTCjXUa40Q
3  4rP96apbJCohHkJB4aiBAf  0HjoyITAvSVktTCjXUa40Q
4  1RqJYBleL85OXwqO8Ri82I  0HjoyITAvSVktTCjXUa40Q
234311


/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


234311


In [44]:
# TRACKS - FINAL
# tracks.drop(['album','artist_names','artist_ids'],axis=1,inplace=True)
tracks.rename({'key':'music_key'}, axis=1, inplace=True)
print(tracks.columns)
print(len(tracks))
tracks.drop_duplicates(inplace=True)
# print(max(tracks.groupby(tracks.columns.tolist(),as_index=False).size()))

print(tracks.head())
print(tracks[tracks['track_id']=="00H03yK4yVXiD1kYlMxChV"])
tracks["explicit"] = tracks["explicit"].apply(lambda val: 0 if val==False else (1 if val==True else 2))
print(len(tracks))
td = tracks.drop_duplicates(subset=['track_id'])
print(len(td))
print(tracks.head())
print(tracks[tracks['track_id'].isnull()])
tracks.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/tracks_new.csv')
# Index(['track_id', 'track_name', 'album', 'album_id', 'artist_names',
#        'artist_ids', 'explicit', 'danceability', 'energy', 'key', 'loudness',
#        'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
#        'valence', 'tempo', 'duration_ms', 'release_date'],
#       dtype='object')

Index(['track_id', 'track_name', 'album_id', 'explicit', 'danceability',
       'energy', 'music_key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'release_date'],
      dtype='object')
1204025
                 track_id             track_name                album_id  \
0  7lmeHLHBe4nmXzuXc0HDjk                Testify  2eia0myWFgoHuttJytCxgX   
1  1wsRitfRRtWyEapl0q22o8        Guerrilla Radio  2eia0myWFgoHuttJytCxgX   
2  1hR0fIFK2qRG3f3RF70pb7       Calm Like a Bomb  2eia0myWFgoHuttJytCxgX   
3  2lbASgTSoDO7MTuLAXlTW0              Mic Check  2eia0myWFgoHuttJytCxgX   
4  1MQTmpYOZ6fcMQc56Hdo7T  Sleep Now In the Fire  2eia0myWFgoHuttJytCxgX   

   explicit  danceability  energy  music_key  loudness  mode  speechiness  \
0         0         0.470   0.978          7    -5.399     1       0.0727   
1         1         0.599   0.957         11    -5.764     1       0.1880   
2         0         0.315   0.

In [64]:
# Find tracks whose release-date consists of only a year
tracksYr = tracks.loc[tracks['release_date'].str.len() == 4]
print(tracksYr.head())
print(tracks.head())
print(tracks.dtypes)
tracksYr['release_date'] = tracksYr['release_date'].apply(lambda yr: str(yr) + "-01-01")
print(tracksYr.dtypes)
print(tracksYr.head())
print(len(tracksYr))
tracksYr.drop_duplicates(inplace=True)
print(len(tracksYr))
tracksYr.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/tracks_failed_date_reformatted.csv')

                   track_id     track_name                album_id  explicit  \
169  45ZjRrEFgHJhCQ72qKDeem       Above Me  0mKXj9hXclmhvassTP0LCw         0   
170  4gd1XbH3adOCB078KP7Dbd          Still  0mKXj9hXclmhvassTP0LCw         0   
171  48LAwhnuZb4un5EdUluL8K  One Slowdance  0mKXj9hXclmhvassTP0LCw         0   
172  4rP96apbJCohHkJB4aiBAf     In My Eyes  0mKXj9hXclmhvassTP0LCw         0   
173  1RqJYBleL85OXwqO8Ri82I      She Cries  0mKXj9hXclmhvassTP0LCw         0   

     danceability  energy  music_key  loudness  mode  speechiness  \
169         0.319   0.948          5    -4.981     1       0.0658   
170         0.286   0.980          9    -4.561     1       0.1050   
171         0.478   0.757         11    -5.093     1       0.0527   
172         0.342   0.968          7    -4.591     0       0.0939   
173         0.279   0.916          9    -5.314     1       0.0824   

     acousticness  instrumentalness  liveness  valence    tempo  duration_ms  \
169      0.000189       

<ipython-input-64-48523e24361c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracksYr['release_date'] = tracksYr['release_date'].apply(lambda yr: str(yr) + "-01-01")
/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


130144


In [ ]:
genres_notnull = genres[genres['genre'].notnull()]
print(len(genres[genres['genre'].isnull()]))
print(len(genres_notnull))
genres_notnull.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/genres_nonull.csv')

17496
5481


In [ ]:
print(tracks['music_key'].describe())




count    1.204025e+06
mean     5.194151e+00
std      3.536731e+00
min      0.000000e+00
25%      2.000000e+00
50%      5.000000e+00
75%      8.000000e+00
max      1.100000e+01
Name: music_key, dtype: float64


In [ ]:
# Artists: mbid -> spotify id (from songBy, which comes from tracks so uses spotify ids)
artists_spotify = sb_art[['artist_id_x','artist_name','listeners','country','genres']]
artists_spotify.rename({'artist_id_x':'artist_id'}, axis=1, inplace=True)
artists_spotify.drop_duplicates(['artist_id'], inplace=True)


# Get genres from artists
print(artists_spotify['genres'].head())
print(artists_spotify.columns)
genres = artists_spotify[['artist_id', 'genres']]
genres['genres'] = genres['genres'].apply(lambda s: [gen for gen in s.replace(" ", "").split(';')] if not pd.isnull(s) else s)
genres.rename({'genres':'genre'}, axis=1, inplace=True)
genres_expl = genres.explode('genre')
genres_expl = genres_expl[genres_expl['genre'].notnull()]
genres_expl.drop_duplicates(inplace=True)
print(genres_expl.head())
print(len(genres_expl))

genres_expl['genre'] = genres_expl['genre'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
genres_expl.drop_duplicates(inplace=True)
genres_expl = genres_expl[genres_expl['genre'].notnull()]
genres_expl = genres_expl.loc[genres_expl['genre'].str.len() > 0]
print(genres_expl.head())
print(len(genres_expl))

print(genres_expl.loc[genres_expl['genre'].isnull()])

genres_expl.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/genres.csv')



artists_spotify.drop('genres', axis=1, inplace=True)
print(artists_spotify.head())
print(len(artists_spotify))
artists_spotify.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/artists_new.csv')

In [ ]:
print(len(genres_expl[genres_expl['genre'].isnull()]))
print(len(genres_expl))
genres_expl.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/genres.csv')

0
19505


In [ ]:
print(sb_expl.iloc[305:310])

                   track_id              artist_ids  \
299  7jWbXvrgdbkajU8L28ahn5  6kACVPfCOnqzgfEF5ryl0x   
299  7jWbXvrgdbkajU8L28ahn5  7wCjDgV6nqBsHguQXPAaIM   
299  7jWbXvrgdbkajU8L28ahn5  0vYQRW5LIDeYQOccTviQNX   
300  2zfK1yLW1HzHGbmYx6X9F8  6e7QpHYqEiyJGiM98IysLa   
300  2zfK1yLW1HzHGbmYx6X9F8  5W5bDNCqJ1jbCgTxDD0Cb3   

                                          artist_names  
299  ['The Highwaymen', 'Willie Nelson', 'Johnny Ca...  
299  ['The Highwaymen', 'Willie Nelson', 'Johnny Ca...  
299  ['The Highwaymen', 'Willie Nelson', 'Johnny Ca...  
300  ['The Highwaymen', 'Willie Nelson', 'Johnny Ca...  
300  ['The Highwaymen', 'Willie Nelson', 'Johnny Ca...  


In [ ]:
# tracks.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/tracksDf.csv')
# artists.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/artistsDf.csv')
# albums.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/albumsDf.csv')
# songBy.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/songByDf.csv')
# albumBy.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/albumByDf.csv')

In [ ]:
tracksDf = pd.read_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/tracksDf.csv')

In [ ]:
# Convert "False" and "True" to 0 and 1 in tracksDf
# tracksDf["explicit"] = tracksDf["explicit"].apply(lambda val: 0 if val==False else (1 if val==True else 2))
# print(tracksDf.head())
tracksDf.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/tracksDf_intbools.csv')

In [ ]:
# songBy = pd.read_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/songByDf.csv')
# artists = pd.read_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/artistsDf.csv')
print(artists.head())
print(artists.columns)

   Unnamed: 0                             artist_id            artist_name  \
0           0  cc197bad-dc9c-440d-a5b5-d52ba2e14234               Coldplay   
1           1  a74b1b7f-71a5-4011-9441-d0b5e4122711              Radiohead   
2           2  8bfac288-ccc5-448d-9573-c33ea2aa5c30  Red Hot Chili Peppers   
3           3  73e5e69d-3554-40d8-8516-00cb38737a1c                Rihanna   
4           4  b95ce3ff-3d05-4e87-9e01-c97b66af13d4                 Eminem   

   listeners    scrobbles         country  \
0  5381567.0  360111850.0  United Kingdom   
1  4732528.0  499548797.0  United Kingdom   
2  4620835.0  293784041.0   United States   
3  4558193.0  199248986.0   United States   
4  4517997.0  199507511.0   United States   

                                              genres  
0  rock; pop; alternative rock; british; uk; brit...  
1  rock; electronic; alternative rock; british; g...  
2  rock; alternative rock; 80s; 90s; rap; metal; ...  
3  pop; dance; hip hop; reggae; contempo

In [ ]:
# songBy = songBy.drop('artist_names', axis=1);
# print(songBy.head(50))
print(songBy[songBy['artist_ids'].str.contains(',')].head())

     Unnamed: 0                track_id  \
236         236  5SZ5EOiHFNfyAu4pFrOtHh   
243         243  0EPb4ruCwyPqt3FA70JhUV   
299         299  7jWbXvrgdbkajU8L28ahn5   
300         300  2zfK1yLW1HzHGbmYx6X9F8   
301         301  7mlWXXM4qSF3faqxmc2XuM   

                                            artist_ids  
236  ['0m3R4g8wdmgu5YByHlrmQX', '6VpZD6SRjh5ZurP9tw...  
243  ['4WivN6D56Y1AtjWPQ0HRRt', '0m3R4g8wdmgu5YByHl...  
299  ['6e7QpHYqEiyJGiM98IysLa', '5W5bDNCqJ1jbCgTxDD...  
300  ['6e7QpHYqEiyJGiM98IysLa', '5W5bDNCqJ1jbCgTxDD...  
301  ['6e7QpHYqEiyJGiM98IysLa', '5W5bDNCqJ1jbCgTxDD...  


In [ ]:
songBy = pd.read_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/songByDf_ids_only.csv')
print(songBy.head())

   Unnamed: 0  Unnamed: 0.1                track_id  \
0           0             0  7lmeHLHBe4nmXzuXc0HDjk   
1           1             1  1wsRitfRRtWyEapl0q22o8   
2           2             2  1hR0fIFK2qRG3f3RF70pb7   
3           3             3  2lbASgTSoDO7MTuLAXlTW0   
4           4             4  1MQTmpYOZ6fcMQc56Hdo7T   

                   artist_ids  
0  ['2d0hyoQ5ynDBnkvAbJKORj']  
1  ['2d0hyoQ5ynDBnkvAbJKORj']  
2  ['2d0hyoQ5ynDBnkvAbJKORj']  
3  ['2d0hyoQ5ynDBnkvAbJKORj']  
4  ['2d0hyoQ5ynDBnkvAbJKORj']  


In [ ]:
# SongBy with one line per artist id (possibly multiple lines per song)
# songBySplit = songBy.explode('artist_ids')
# print(songBy.head())
songBySplit= songBy.set_index(['track_id'])['artist_ids'].apply(pd.Series).stack()
songBySplit = songBySplit.reset_index()
# songBySplit.columns = ['track_id','artist_id']
print(songBySplit.head(30))
# print(songBySplit[songBySplit['artist_ids'].str.contains(',')].head(30))

                  track_id  level_1                           0
0   7lmeHLHBe4nmXzuXc0HDjk        0  ['2d0hyoQ5ynDBnkvAbJKORj']
1   1wsRitfRRtWyEapl0q22o8        0  ['2d0hyoQ5ynDBnkvAbJKORj']
2   1hR0fIFK2qRG3f3RF70pb7        0  ['2d0hyoQ5ynDBnkvAbJKORj']
3   2lbASgTSoDO7MTuLAXlTW0        0  ['2d0hyoQ5ynDBnkvAbJKORj']
4   1MQTmpYOZ6fcMQc56Hdo7T        0  ['2d0hyoQ5ynDBnkvAbJKORj']
5   2LXPNLSMAauNJfnC58lSqY        0  ['2d0hyoQ5ynDBnkvAbJKORj']
6   3moeHk8eIajvUEzVocXukf        0  ['2d0hyoQ5ynDBnkvAbJKORj']
7   4llunZfVXv3NvUzXVB3VVL        0  ['2d0hyoQ5ynDBnkvAbJKORj']
8   21Mq0NzFoVRvOmLTOnJjng        0  ['2d0hyoQ5ynDBnkvAbJKORj']
9   6s2FgJbnnMwFTpWJZzvb6z        0  ['2d0hyoQ5ynDBnkvAbJKORj']
10  7o2Razxnl33Jrgz4PpMYNa        0  ['2d0hyoQ5ynDBnkvAbJKORj']
11  3Hxu3Ey1TV9mZ5XqMXckiS        0  ['2d0hyoQ5ynDBnkvAbJKORj']
12  6ZU9RJIZ0fNaFuQM57bDIA        0  ['2d0hyoQ5ynDBnkvAbJKORj']
13  3FUS56gKr9mVBmzvlnodlh        0  ['2d0hyoQ5ynDBnkvAbJKORj']
14  3tTL7jlSkowXidYeafFtwG        0  ['2

In [ ]:
# lst_col = 'artist_ids'
# col = 'track_id'
# print(songBy[lst_col].values)
sb = songBy[['track_id', 'artist_ids']]
sb = sb.astype({'artist_ids': '[str]'}).dtypes
sb['artist_ids'] = sb['artist_ids'].apply(lambda lst: str(lst).replace("\'", ""))
print(sb.explode('artist_ids'))
print(sb[sb['artist_ids'].str.contains(",")])

# songBySplit = pd.DataFrame({
#       col:np.repeat(songBy[col].values, songBy[lst_col].str.len())
#       for col in songBy.columns.drop(lst_col)}
#     ).assign(**{lst_col:np.concatenate(songBy[lst_col].values)})[songBy.columns]

In [ ]:
sb = songBy.set_index(['track_id']).apply(pd.Series.explode).reset_index()
print(sb.head())

                 track_id  Unnamed: 0  Unnamed: 0.1  \
0  7lmeHLHBe4nmXzuXc0HDjk           0             0   
1  1wsRitfRRtWyEapl0q22o8           1             1   
2  1hR0fIFK2qRG3f3RF70pb7           2             2   
3  2lbASgTSoDO7MTuLAXlTW0           3             3   
4  1MQTmpYOZ6fcMQc56Hdo7T           4             4   

                   artist_ids  
0  ['2d0hyoQ5ynDBnkvAbJKORj']  
1  ['2d0hyoQ5ynDBnkvAbJKORj']  
2  ['2d0hyoQ5ynDBnkvAbJKORj']  
3  ['2d0hyoQ5ynDBnkvAbJKORj']  
4  ['2d0hyoQ5ynDBnkvAbJKORj']  


In [ ]:
sb = songBy.set_index(['track_id']).apply(pd.Series.explode).reset_index()
print(sb.head())
print(sb.explode('artist_ids').head())
sb_split = sb.explode('artist_ids')
sb.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/songByDf_ids_only_split.csv')

                 track_id              artist_ids
0  7lmeHLHBe4nmXzuXc0HDjk  2d0hyoQ5ynDBnkvAbJKORj
1  1wsRitfRRtWyEapl0q22o8  2d0hyoQ5ynDBnkvAbJKORj
2  1hR0fIFK2qRG3f3RF70pb7  2d0hyoQ5ynDBnkvAbJKORj
3  2lbASgTSoDO7MTuLAXlTW0  2d0hyoQ5ynDBnkvAbJKORj
4  1MQTmpYOZ6fcMQc56Hdo7T  2d0hyoQ5ynDBnkvAbJKORj


In [ ]:
sb_split.rename({'artist_ids': 'artist_id'}, axis=1, inplace=True)
print(sb_split.head())
sb_split.to_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/songByDf_ids_only_split_explode.csv')

                 track_id               artist_id
0  7lmeHLHBe4nmXzuXc0HDjk  2d0hyoQ5ynDBnkvAbJKORj
1  1wsRitfRRtWyEapl0q22o8  2d0hyoQ5ynDBnkvAbJKORj
2  1hR0fIFK2qRG3f3RF70pb7  2d0hyoQ5ynDBnkvAbJKORj
3  2lbASgTSoDO7MTuLAXlTW0  2d0hyoQ5ynDBnkvAbJKORj
4  1MQTmpYOZ6fcMQc56Hdo7T  2d0hyoQ5ynDBnkvAbJKORj


In [ ]:
print(sb_split.describe())

                      track_id               artist_id
count                  1798207                 1798207
unique                 1204025                  140449
top     3lZ5reS0bAq9REusvBOeql  5aIqB5nVVvmFsvSdExz408
freq                        51                    9708


In [ ]:
tracks = pd.read_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/tracksDf.csv')

In [ ]:
artists = pd.read_csv('/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/artistsDf.csv')

In [ ]:
# sb_art = sb_split.merge(artists, on='artist_id', how='inner')
# print(sb_art.head())
print("SONGS:")
print(tracks.columns)
print(tracks[['track_id','album_id','artist_ids']].head())
print("ARTISTS:")
print(artists.head())

SONGS:
Index(['Unnamed: 0', 'track_id', 'track_name', 'album_id', 'explicit',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'release_date'],
      dtype='object')


KeyError: ignored

In [ ]:
tracks_path = '/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/tracks.csv'
artists_path = '/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/artists.csv'

orig_tracks = pd.read_csv(tracks_path)
orig_artists = pd.read_csv(artists_path)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
print(orig_tracks.columns)
print(orig_artists.columns)

Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date'],
      dtype='object')
Index(['mbid', 'artist_mb', 'artist_lastfm', 'country_mb', 'country_lastfm',
       'tags_mb', 'tags_lastfm', 'listeners_lastfm', 'scrobbles_lastfm',
       'ambiguous_artist'],
      dtype='object')


In [ ]:
print(orig_tracks[['artist_ids','artists']].head())
print(orig_artists[['mbid','artist_mb']].head())


                   artist_ids                       artists
0  ['2d0hyoQ5ynDBnkvAbJKORj']  ['Rage Against The Machine']
1  ['2d0hyoQ5ynDBnkvAbJKORj']  ['Rage Against The Machine']
2  ['2d0hyoQ5ynDBnkvAbJKORj']  ['Rage Against The Machine']
3  ['2d0hyoQ5ynDBnkvAbJKORj']  ['Rage Against The Machine']
4  ['2d0hyoQ5ynDBnkvAbJKORj']  ['Rage Against The Machine']
                                   mbid              artist_mb
0  cc197bad-dc9c-440d-a5b5-d52ba2e14234               Coldplay
1  a74b1b7f-71a5-4011-9441-d0b5e4122711              Radiohead
2  8bfac288-ccc5-448d-9573-c33ea2aa5c30  Red Hot Chili Peppers
3  73e5e69d-3554-40d8-8516-00cb38737a1c                Rihanna
4  b95ce3ff-3d05-4e87-9e01-c97b66af13d4                 Eminem


In [ ]:
print(artists.columns)

Index(['Unnamed: 0', 'artist_id', 'artist_name', 'listeners', 'scrobbles',
       'country', 'genres'],
      dtype='object')


In [ ]:
tdf = sb_split.merge(songBy, how='inner')
print(tdf.head())

KeyError: ignored

In [ ]:
sb2 = tracks[["track_id", "artist_ids", "artist_names"]]
sb2_seti = sb2.set_index(['track_id']).apply(pd.Series.explode).reset_index()
print(sb2.head())
print(sb2.explode('artist_ids').head())
sb_split2 = sb2.explode('artist_ids')
print(sb_split2.head())


KeyError: ignored